# Projeto Final
## Matheus Freitas Martins

## Importando dependências 

In [307]:
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

## Funções 

In [308]:
def get_categories(site_id):
    url = f"https://api.mercadolibre.com/sites/{site_id}/categories"
    response = requests.get(url)
    data = response.json()
    return data

def search_items(category_id, offset=0):
    url = "https://api.mercadolibre.com/sites/MLB/search"
    params = {
        "category": category_id,
        "offset": offset  
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data['results']

def get_items_for_categories(category_ids, items_per_category=1000):
    df = pd.DataFrame()  # Dataframe para armazenar os dados

    # Iterando sobre cada ID de categoria na lista
    for category_id in category_ids:
        print(f"Buscando produtos da categoria {category_id}...")
        
        # Como o Mercado Livre retorna um máximo de 50 itens por pedido, precisamos fazer várias solicitações
        for i in range(0, items_per_category, 50):
            # Buscando produtos da categoria
            items = search_items(category_id, offset=i)
            
            # Adicionando a categoria aos itens
            for item in items:
                item['source_category'] = category_id
            
            # Adicionando ao dataframe
            df = pd.concat([df, pd.DataFrame(items)], ignore_index=True)
    
    return df

# Calcula o preço médio, mínimo e máximo para a categoria
def analise_precos(df_categoria):
    resumo_preco = df_categoria['price'].agg(['mean', 'min', 'max'])
    return resumo_preco

# Ordena os produtos pela quantidade vendida e retorna o mais vendido
def analise_popularidade(df_categoria):
    produto_popular = df_categoria.sort_values('sold_quantity', ascending=False).iloc[0]
    return produto_popular['title'], produto_popular['sold_quantity'], produto_popular['price']

def analise_categoria(df, categoria):
    # Filtra o dataframe para a categoria selecionada
    df_categoria = df[df['source_category'] == categoria]
    
    # Executa as análises de preço e popularidade
    resumo_preco = analise_precos(df_categoria)
    nome_produto_popular, vendas_popular, preco_produto_popular = analise_popularidade(df_categoria)
    
    print(f'Categoria: {categoria}')
    print(f'Preço médio: {resumo_preco["mean"]:.2f}')
    print(f'Preço mínimo: {resumo_preco["min"]:.2f}')
    print(f'Preço máximo: {resumo_preco["max"]:.2f}')
    print(f'Produto mais popular: {nome_produto_popular}')
    print(f'Quantidade vendida do produto mais popular: {vendas_popular}')
    print(f'Preço do produto mais popular: {preco_produto_popular:.2f}')
    print('\n')

In [309]:
site_id = 'MLB'  # ID do site do Mercado Livre Brasil
categories = get_categories(site_id)

# Imprime o ID e o nome de todas as categorias
for category in categories:
    print(f"ID: {category['id']}, Nome: {category['name']}")

ID: MLB5672, Nome: Acessórios para Veículos
ID: MLB271599, Nome: Agro
ID: MLB1403, Nome: Alimentos e Bebidas
ID: MLB1071, Nome: Animais
ID: MLB1367, Nome: Antiguidades e Coleções
ID: MLB1368, Nome: Arte, Papelaria e Armarinho
ID: MLB1384, Nome: Bebês
ID: MLB1246, Nome: Beleza e Cuidado Pessoal
ID: MLB1132, Nome: Brinquedos e Hobbies
ID: MLB1430, Nome: Calçados, Roupas e Bolsas
ID: MLB1039, Nome: Câmeras e Acessórios
ID: MLB1743, Nome: Carros, Motos e Outros
ID: MLB1574, Nome: Casa, Móveis e Decoração
ID: MLB1051, Nome: Celulares e Telefones
ID: MLB1500, Nome: Construção
ID: MLB5726, Nome: Eletrodomésticos
ID: MLB1000, Nome: Eletrônicos, Áudio e Vídeo
ID: MLB1276, Nome: Esportes e Fitness
ID: MLB263532, Nome: Ferramentas
ID: MLB12404, Nome: Festas e Lembrancinhas
ID: MLB1144, Nome: Games
ID: MLB1459, Nome: Imóveis
ID: MLB1499, Nome: Indústria e Comércio
ID: MLB1648, Nome: Informática
ID: MLB218519, Nome: Ingressos
ID: MLB1182, Nome: Instrumentos Musicais
ID: MLB3937, Nome: Joias e Relóg

## Lista dos IDs de categorias desejadas

In [310]:
#desired_categories = ['MLB1384', 'MLB1246', 'MLB1132', 'MLB1430', 'MLB3937', 'MLB1648', 'MLB1051', 'MLB1574', 'MLB12404', 'MLB1000', 'MLB1276']  # Insira seus IDs de categoria desejados aqui

## Criar dataframe com base nas categorias escolhidas

In [311]:
#df = get_items_for_categories(desired_categories)

## Salvar o DataFrame em um arquivo CSV

In [312]:
#df.to_csv('mercado_livre_data.csv', index=False)

## Entendendo os dados

In [313]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10948 entries, 0 to 10947
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price               10948 non-null  float64
 1   available_quantity  10948 non-null  int64  
 2   condition           10948 non-null  object 
 3   title               10948 non-null  object 
 4   source_category     10948 non-null  object 
 5   sold_quantity       10948 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 513.3+ KB


In [314]:
df.head(5)

,price,available_quantity,condition,title,source_category,sold_quantity
0,20.90,1,new,Shampoo Regular Johnson's Baby 200ml,MLB1384,500
1,124.90,150,new,Pampers Limpeza Delicada Lenços Umedecidos 576 Un,MLB1384,500
2,88.04,500,new,Fraldas Pampers Confort Sec G 70 U,MLB1384,5000
3,81.61,500,new,Kit 21 Pçs Maternidade Roupa De Bebê Frete Gratis,MLB1384,5000
4,19.49,1,new,Meia Bebê Com 3 Pares De Algodão Confortável B...,MLB1384,50


In [315]:
unique_categories = df['source_category'].unique()
print(unique_categories)

['MLB1384' 'MLB1246' 'MLB1132' 'MLB1430' 'MLB3937' 'MLB1648' 'MLB1051'
 'MLB1574' 'MLB12404' 'MLB1000' 'MLB1276']


## Quantidade de elementos em cada categoria

In [316]:
category_counts = df['source_category'].value_counts()
print(category_counts)

MLB12404    1000
MLB1276     1000
MLB1384      999
MLB1246      998
MLB1132      998
MLB1430      998
MLB3937      998
MLB1000      998
MLB1648      996
MLB1051      994
MLB1574      969
Name: source_category, dtype: int64


## Carregando os dados

In [317]:
df = pd.read_csv("mercado_livre_data.csv")

## Categoria com o maior número total de vendas.

In [318]:
df['sold_quantity'] = df['sold_quantity'].astype(int)
sales_by_category = df.groupby('source_category')['sold_quantity'].sum()
max_sales_category = sales_by_category.idxmax()

print(f"A categoria com o maior número de vendas é: {max_sales_category}")


A categoria com o maior número de vendas é: MLB1574


## Obtendo informações relevantes sobre cada categoria

In [319]:
categorias = df['source_category'].unique()

for categoria in categorias:
    analise_categoria(df, categoria)

Categoria: MLB1384
Preço médio: 127.06
Preço mínimo: 5.00
Preço máximo: 3999.99
Produto mais popular: 10 Trava Gaveta Armário Segurança Protetor Proteção Bebe Inc
Quantidade vendida do produto mais popular: 50000
Preço do produto mais popular: 29.90


Categoria: MLB1246
Preço médio: 109.66
Preço mínimo: 2.36
Preço máximo: 999.99
Produto mais popular: Touca Descartável Tnt C/ Elástico C/100 C/ Nota
Quantidade vendida do produto mais popular: 50000
Preço do produto mais popular: 15.50


Categoria: MLB1132
Preço médio: 145.59
Preço mínimo: 9.77
Preço máximo: 2542.50
Produto mais popular: Micro Servo Motor 9g Sg90 Kit Com 4 Unidades - Nfe
Quantidade vendida do produto mais popular: 5000
Preço do produto mais popular: 44.90


Categoria: MLB1430
Preço médio: 117.50
Preço mínimo: 7.00
Preço máximo: 3600.00
Produto mais popular: Meias Masculino Kit 12 Pares Esportes Atacado Revenda Oferta
Quantidade vendida do produto mais popular: 50000
Preço do produto mais popular: 34.11


Categoria: MLB393

## Treinando o modelo para prever quantidade de vendas

In [320]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10948 entries, 0 to 10947
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10948 non-null  object 
 1   title                 10948 non-null  object 
 2   condition             10948 non-null  object 
 3   thumbnail_id          10948 non-null  object 
 4   catalog_product_id    6154 non-null   object 
 5   listing_type_id       10948 non-null  object 
 6   permalink             10948 non-null  object 
 7   buying_mode           10948 non-null  object 
 8   site_id               10948 non-null  object 
 9   category_id           10948 non-null  object 
 10  domain_id             10948 non-null  object 
 11  thumbnail             10948 non-null  object 
 12  currency_id           10948 non-null  object 
 13  order_backend         10948 non-null  int64  
 14  price                 10948 non-null  float64
 15  original_price     

In [321]:
features = ['price', 'available_quantity', 'condition']

for feature in features:
    if df[feature].dtype == 'object':
        df[feature] = df[feature].fillna('missing')
    else:
        df[feature] = df[feature].fillna(0)

X = df[features]
y = df['sold_quantity']

# Pré-processamento para variáveis categóricas e númericas
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, ['price', 'available_quantity']),
        ('cat', categorical_transformer, ['condition'])])

model = RandomForestRegressor(n_estimators=100, random_state=0)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf.fit(X_train, y_train)

# Fazendo previsões
preds = clf.predict(X_test)

# Avaliando o modelo
rmse = mean_squared_error(y_test, preds, squared=False)
print('RMSE:', rmse)

RMSE: 1947.2592363419699


# Teste de previsão

In [322]:
test_data = {
    'price': [100.0],
    'available_quantity': [10],
    'condition': ['new']
}

df_test = pd.DataFrame(test_data)
pred_test = clf.predict(df_test)
print('Previsão de quantidade vendida:', pred_test)


Previsão de quantidade vendida: [120.64166565]
